### 0. Install dependencies

In [ ]:
%pip install -qU pip
%pip install -qU sagemaker boto3 awscli boto3 ipywidgets

In [ ]:
import json
from pathlib import Path
import os
import sys
from time import strftime
from functools import partial
import importlib

# if the following command fails you may need to modify
# it to match the notebook's CWD:
!cp ../utilities.py scripts
sys.path.append('../')
import utilities as u

import boto3
import sagemaker
from sagemaker.experiments.run import Run
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig

In [ ]:
sagemaker.__version__, boto3.__version__

In [ ]:
boto_session = boto3.session.Session()
sagemaker_session = sagemaker.session.Session(boto_session)
omics = boto3.client("omics")

REGION_NAME = sagemaker_session.boto_region_name
S3_BUCKET = sagemaker_session.default_bucket()

EXPERIMENT_NAME = "hyenaDNA-pretraining-v2"

SAGEMAKER_EXECUTION_ROLE = sagemaker.session.get_execution_role(sagemaker_session)
print(f"Assumed SageMaker role is {SAGEMAKER_EXECUTION_ROLE}")

### 1. Read the data from AWS HealthOmics

To train the HyenaDNA model, we will directly utilize the data from the Healthomics Sequence Store. Let's retrieve the read set URI from the Healthomics Sequence Store. Make sure to replace your sequence store id in the following cell. 

In [ ]:
seq_store_id = "4789217446"  # replace with your sequence store id

In [ ]:
seq_store_info = omics.get_sequence_store(id=seq_store_id)
s3_uri = seq_store_info["s3Access"]["s3Uri"]
s3_arn = seq_store_info["s3Access"]["s3AccessPointArn"]
key_arn = seq_store_info["sseConfig"]["keyArn"]
s3_uri, s3_arn, key_arn

In [ ]:
S3_DATA_URI = f"{s3_uri}readSet/"
S3_DATA_URI

For the training job to access the objects in the above S3 access point, `s3uri`, you must add a policy
to this execution role (`SAGEMAKER_EXECUTION_ROLE`). The output of the following cell is the policy that
you should attach to this role:

In [ ]:
print(json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        { 
            "Sid": "S3DirectAccess",
            "Effect": "Allow",
            "Action": [ 
                "s3:GetObject",
                "s3:ListBucket"
            ], 
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "s3:DataAccessPointArn": s3_arn
                } 
            }
        },
        { 
            "Sid": "DefaultSequenceStoreKMSDecrypt",
            "Effect": "Allow",
            "Action": "kms:Decrypt",
            "Resource": key_arn
        }
    ] 
}, indent=2))

### 2. Training



### 2.1 Define the training container 

In [ ]:
pytorch_image_uri = f"763104351884.dkr.ecr.{REGION_NAME}.amazonaws.com/pytorch-training:2.2.0-gpu-py310-cu121-ubuntu20.04-sagemaker"
pytorch_image_uri

#### 2.2 Define the training job parameters

In [ ]:
MODEL_ID = 'LongSafari/hyenadna-small-32k-seqlen-hf'
TRAINING_JOB_NAME = 'hyenaDNA-pretraining'

# Additional training parameters
hyperparameters = {
    "species" : "mouse",
    "epochs": 150,
    "model_checkpoint": MODEL_ID,
    "max_length": 32_000,
    "batch_size": 4, 
    "learning_rate": 6e-4,
    "weight_decay" : 0.1,
    "log_level" : "INFO",
    "log_interval" : 100
}


#### 2.3 Define Metrics to track


In [ ]:
metric_definitions = [
    {"Name": "epoch", "Regex": "Epoch: ([0-9.]*)"},
    {"Name": "step", "Regex": "Step: ([0-9.]*)"},
    {"Name": "train_loss", "Regex": "Train Loss: ([0-9.e-]*)"},
    {"Name": "train_perplexity", "Regex": "Train Perplexity: ([0-9.e-]*)"},
    {"Name": "eval_loss", "Regex": "Eval Average Loss: ([0-9.e-]*)"},
    {"Name": "eval_perplexity", "Regex": "Eval Perplexity: ([0-9.e-]*)"}
]

#### 2.4 Define the tensorboard configurations to track the training results

In [ ]:
LOG_DIR="/opt/ml/output/tensorboard"

output_path = os.path.join(
    "s3://", S3_BUCKET, "sagemaker-output", "training", TRAINING_JOB_NAME
)

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=os.path.join(output_path, 'tensorboard'),
    container_local_output_path=LOG_DIR
)

#### 2.4 Define Estimator

In [ ]:
hyenaDNA_estimator = PyTorch(
    base_job_name=TRAINING_JOB_NAME,
    entry_point="train_hf_accelerate.py",
    source_dir="scripts/",
    instance_type="ml.g5.12xlarge",
    instance_count=1,
    image_uri=pytorch_image_uri,
    role=SAGEMAKER_EXECUTION_ROLE,
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    sagemaker_session=sagemaker_session,
    distribution={"torch_distributed": {"enabled": True}},
    tags=[{"Key": "project", "Value": "genomics-model-pretraining"}],
    keep_alive_period_in_seconds=1800,
    tensorboard_output_config=tensorboard_output_config,
)


#### 2.5 Start Training with Distributed Data Parallel

In [ ]:
with Run(
    experiment_name=EXPERIMENT_NAME,
    sagemaker_session=sagemaker_session,
) as run:
    hyenaDNA_estimator.fit(
        {
            "data": TrainingInput(
                s3_data=S3_DATA_URI, input_mode="File"
            ),
        },
        wait=True,
    )


In [ ]:
training_job_name = hyenaDNA_estimator.latest_training_job.name
training_job_name

### 5. Training Results 

* In our training process we had pushed the training resulsts to Tensorboard. You can see them using SageMaker tensorboad application. To obtain the link to the tensorboard, please replace the sagemaker studio profile name below. 

In [ ]:
user_profile = "shamika" # replace with your sagemaker studio profile name

Execuate following cell to get link to the the tensorboard application

In [ ]:
from sagemaker.interactive_apps.tensorboard import TensorBoardApp

with open("/opt/ml/metadata/resource-metadata.json", "r") as f:
    app_metadata = json.loads(f.read())
    sm_user_profile_name = app_metadata["SpaceName"]
    sm_domain_id = app_metadata["DomainId"]

tb_app = TensorBoardApp(REGION_NAME)
tb_app.get_app_url(
    training_job_name=training_job_name,
    create_presigned_domain_url=True,           
    domain_id=sm_domain_id,                 
    user_profile_name=user_profile, 
    open_in_default_web_browser=False,
    optional_create_presigned_url_kwargs={} 
)

### 6. Deploy trained model to an realtime endpoint

In [ ]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.estimator import Estimator

training_job_name = "hyenaDNA-pretraining-2024-04-06-06-23-26-412"
#attached_estimator = Estimator.attach(training_job_name)

model_data = attached_estimator.model_data
model_data

In [ ]:
# Deploy the model to create a real-time endpoint
endpoint_name = 'hyenaDNA-pretrained-mouse-ep'  
pytorch_deployment_uri = f"763104351884.dkr.ecr.{REGION_NAME}.amazonaws.com/pytorch-inference:2.2.0-gpu-py310-cu118-ubuntu20.04-sagemaker"

hyenaDNAModel = PyTorchModel(
    model_data=model_data,
    role=SAGEMAKER_EXECUTION_ROLE,
    image_uri=pytorch_deployment_uri,
    entry_point="inference.py",
    source_dir="scripts/",
    sagemaker_session=sagemaker_session,
    name=endpoint_name,
    env = {
        'MMS_MAX_REQUEST_SIZE': '2000000000',
        'MMS_MAX_RESPONSE_SIZE': '2000000000',
        'MMS_DEFAULT_RESPONSE_TIMEOUT': '900',
        'TS_MAX_RESPONSE_SIZE':'2000000000',
        'TS_MAX_REQUEST_SIZE':'2000000000',
    }
)

In [ ]:
#real_time_endpoint_name = "hyenaDNA-mouse-pretrained-real-ep"
env = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'7200', 
    'TS_MAX_RESPONSE_SIZE':'2000000000',
    'TS_MAX_REQUEST_SIZE':'2000000000',
    'MMS_MAX_RESPONSE_SIZE':'2000000000',
    'MMS_MAX_REQUEST_SIZE':'2000000000'
}

# deploy the endpoint endpoint
realtime_predictor = hyenaDNAModel.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.8xlarge",
    endpoint_name=endpoint_name,
    env=env,
)

### 7. Test the realtime endpoint



In [ ]:
import json
sample_genome_data = []
with open("./sample_mouse_data.json") as file:
    for line in file:
        sample_genome_data.append(json.loads(line))
len(sample_genome_data)

In [ ]:
len(sample_genome_data[0])

In [ ]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer


data = [sample_genome_data[0]]
realtime_predictor.serializer = JSONSerializer()
realtime_predictor.deserializer = JSONDeserializer()
embeddings = realtime_predictor.predict(data=data)

In [ ]:
embeddings

#### 7. Cleanup



In [ ]:
realtime_predictor.delete_endpoint()